# Neural Distinguisher of Reduced Round Speck 32/64

## Speck 32/64

Speck 32/64의 구현이 맞는지 테스트벡터를 우선 확인해보겠습니다.

In [1]:
from speck import Speck

cipher = Speck()
cipher.check_testvector()

10090811101918
Testvector verified.


True

## Neural Distinguisher

Speck32/64의 5, 6, 7라운드 distinguisher의 모델을 생성하고 학습시켜보겠습니다. 

사용한 모델은 다음과 같습니다.자세한 코드는 train.py를 참고하면 됩니다.

- 은닉계층이 2개인 DNN
- Residual Block이 하나인 ResNet
- Residual Block이 10개인 ResNet

시간관계상 epoch를 1로 설정했습니다. 

매 epoch를 진행하면서 validation accuracy 가 가장 높은 상태를 .h5 확장자 파일로 저장합니다.

In [2]:
import train

train.train_all(epochs=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________
Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 17s 2us/sample - loss: 0

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 41s 4us/sample - loss: 0.0859 - acc: 0.8967 - val_loss: 0.0728 - val_acc: 0.9135
Best validaation accuracy:  0.913503
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 4, 16)        0           input_5[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 16, 4)        0           reshape_1[0][0]                  
________________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 41s 4us/sample - loss: 0.2524 - acc: 0.5182 - val_loss: 0.2453 - val_acc: 0.5639
Best validaation accuracy:  0.563859
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 4, 16)        0           input_7[0][0]                    
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 16, 4)        0           reshape_3[0][0]                  
________________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 165s 16us/sample - loss: 0.0806 - acc: 0.9057 - val_loss: 0.0710 - val_acc: 0.9142
Best validaation accuracy:  0.914178
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_4 (Reshape)             (None, 4, 16)        0           input_8[0][0]                    
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 16, 4)        0           reshape_4[0][0]                  
______________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 173s 17us/sample - loss: 0.1769 - acc: 0.7470 - val_loss: 0.1600 - val_acc: 0.7745
Best validaation accuracy:  0.774519
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 4, 16)        0           input_9[0][0]                    
__________________________________________________________________________________________________
permute_5 (Permute)             (None, 16, 4)        0           reshape_5[0][0]                  
______________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 178s 18us/sample - loss: 0.2550 - acc: 0.5118 - val_loss: 0.2502 - val_acc: 0.5421
Best validaation accuracy:  0.542138


epoch이 1이기 때문에 성능이 논문에 적힌 것보다 좋게 나오지 않습니다.

제가 epoch을 200으로 하여 학습시킨 데이터를 trained_models 폴더에 옮겨두었습니다.

해당 모델을 불러들여서 테스트 하는 코드가 evaluate.py에 구현되어 있습니다.

In [3]:
import evaluate

In [4]:
#DNN 결과
evaluate.evaluate_dnn()

Round:  5  Accuracy:  0.885845
Round:  6  Accuracy:  0.692553
Round:  7  Accuracy:  0.521163


In [5]:
#ResNet (block=1) 결과
evaluate.evaluate_resnet(resnet_depth=1)

Round:  5  Accuracy:  0.926429
Round:  6  Accuracy:  0.783671
Round:  7  Accuracy:  0.609289


In [6]:
#ResNet (block=10) 결과
evaluate.evaluate_resnet(resnet_depth=10)

Round:  5  Accuracy:  0.926938
Round:  6  Accuracy:  0.786366
Round:  7  Accuracy:  0.608892


## Key Recovery Attack

5라운드 distinguisher를 사용하여 6라운드 Speck 32/64의 키를 복구해보겠습니다.

해당 코드는 key_recovery.py 에 구현되어 있습니다.

In [7]:
import key_recovery

key_recovery.attack(num_test=5, target_round=6)

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_12 (Reshape)            (None, 4, 16)        0           input_19[0][0]                   
__________________________________________________________________________________________________
permute_12 (Permute)            (None, 16, 4)        0           reshape_12[0][0]                 
__________________________________________________________________________________________________
conv1d_144 (Conv1D)             (None, 16, 32)       160         permute_12[0][0]                 
___________________________________________________________________________________________

D:\workspace\git\neural_distinguisher\key_recovery.py:38: RuntimeWarning: divide by zero encountered in log2
  v = np.log2(v)


Recovery attack ended:  2019-09-11 20:25:19.522058
Elapsed 239.71876001358032
Guessed: 0x3ddc  Diff: 0x0  Rank: 566.8684692382812
Guessed: 0xbddc  Diff: 0x8000  Rank: 564.37255859375
Guessed: 0xfddc  Diff: 0xc000  Rank: 559.7255859375
Guessed: 0x7ddc  Diff: 0x4000  Rank: 558.3466796875
Real key: 0x3ddc  Rank: 566.86847

<<Test 1 >>
Recovery attack started:  2019-09-11 20:25:19.532061
Recovery attack ended:  2019-09-11 20:29:15.867846
Elapsed 236.33578610420227
Guessed: 0xaa6a  Diff: 0x0  Rank: 631.6132202148438
Guessed: 0x2a6a  Diff: 0x8000  Rank: 627.1544189453125
Guessed: 0x6a6a  Diff: 0xc000  Rank: 616.6426391601562
Guessed: 0xea6a  Diff: 0x4000  Rank: 614.3024291992188
Real key: 0xaa6a  Rank: 631.6132

<<Test 2 >>
Recovery attack started:  2019-09-11 20:29:15.873848
Recovery attack ended:  2019-09-11 20:33:09.407025
Elapsed 233.53317666053772
Guessed: 0x6aee  Diff: 0x8000  Rank: 623.7982177734375
Guessed: 0xeaee  Diff: 0x0  Rank: 622.82421875
Guessed: 0x2aee  Diff: 0xc000  Rank: 61